In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-up2i43nx
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-up2i43nx
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpmop9gnrk".


In [4]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1



In [33]:
%%cuda
#include <iostream>
#include <vector>
#include <numeric>
#include <stdio.h>
#include <typeinfo>

__global__ void vectorAdd(float *a, float *b, float *c) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    //printf("%u", a);
    c[i] = a[i] + b[i];
    //printf("%u from %u", c[i], i);
}

int main() {
    dim3 blocksPerGrid(2,1,1);
    dim3 threadsPerBlock(256,1,1);

    int size = 50;

    std::vector<float> a(size);
    std::vector<float> b(size);
    std::vector<float> c(size);

    std::iota(a.begin(), a.end(), 1.0f);
    std::iota(b.begin(), b.end(), 1.0f);

    float* a_dev;
    float* b_dev;
    float* c_dev;

    //cudaMalloc((void**)&a_dev, size * sizeof(float));

    cudaMalloc((void**)&a_dev, size * sizeof(float));
    cudaMalloc((void**)&b_dev, size * sizeof(float));
    cudaMalloc((void**)&c_dev, size * sizeof(float));

    cudaMemcpy(a_dev, a.data(), size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, b.data(), size * sizeof(float), cudaMemcpyHostToDevice);

    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(a_dev, b_dev, c_dev);
    cudaDeviceSynchronize();

    cudaMemcpy(c.data(), c_dev, size * sizeof(float), cudaMemcpyDeviceToHost);


    for (const auto& value : c) {
        std::cout << value << " ";
    }

    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
}

2 4 6 8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44 46 48 50 52 54 56 58 60 62 64 66 68 70 72 74 76 78 80 82 84 86 88 90 92 94 96 98 100 
